## Modeling discount solved for time with a given factor that describes the sum X at a time t in the future

In [1]:
import numpy as np

"""
Given a Price To Sales of 12 - 
How many years is a company priced in the future as their revenue grows by 10 % yearly
"""

# given Conditions
ps = 21 # price to sales
current_revenue = 4.5
g = 0.18 # annual growth rate

$$
S_0 \cdot \alpha = S_0 \cdot (1 + g)^t           
$$ <br>
$$
\alpha = (1 + g)^t             
$$ <br>
$$
log(\alpha) = t \cdot log(1 + g)           
$$ <br>
$$
t = \frac{log(\alpha)}{log(1 + g)} 
$$

In [2]:
t = np.log(ps) / np.log(1 + g)
t

18.394301220651187

Lets simulate this process now as a function f(x), taking an interval $[g_0; g_1]$ for the growth rate as the input, calculating the average time needed for this all rates in the given interval.
An interval describing the annual growth rate fits the real world revenue growth deviations better and gives a better approximation for the time premium payed.

In [3]:
def calculate_time_premium_payed(beta, g_lower, g_upper=None, name=None):
    if g_upper is not None:
        growth_rates = np.arange(g_lower, g_upper + 1, 1)
        
        outputs = []
        
        for g in growth_rates:
            t = np.log(beta) / np.log(1 + g)
            outputs.append(t)
            
        t_mean = np.mean(outputs)
        if t_mean < 0:
            t_mean = np.NaN
        
        print(f"The revenue of {name if name is not None else 'Company X'} trades at a premium of {np.round(t_mean):.0f} years forward at a average growth rate of {np.mean([g_lower, g_upper]) * 100:.2f} %.")
        return t_mean
    
    t = np.log(beta) / np.log(1 + g_lower)
    if t < 0:
        t = np.NaN
        
    print(f"The revenue of {name if name is not None else 'Company X'} trades at a premium of {np.round(t):.0f} years forward at a growth rate of {g_lower * 100:.1f} %.")
    return t

In [4]:
f_x = calculate_time_premium_payed(6, 0.06, 0.12)

The revenue of Company X trades at a premium of 17 years forward at a average growth rate of 9.00 %.


Now let's fetch stock data from the yahoo finance api and implement the model for real world usage

In [5]:
import yfinance as yf

# Note: You could also use yf.Ticker(...).info["priceToSalesTrailing12Months"]

def calculate_price_to_sales(symbol):
    ticker = yf.Ticker(symbol)
    
    name = ticker.info["longName"]
    market_cap = ticker.info["marketCap"]
    revenue = ticker.get_income_stmt(freq="quarterly").loc["TotalRevenue"].sum() # yfinance returns a DataFrame with 4 quarters (=TTM) lookback period
    
    ps = market_cap / revenue
    
    print(f"{name} has a PriceToSalesTTM of {ps:.2f}")
    
    return ps, name

In [6]:
if __name__ == "__main__":
    symbol = "NVDA"
    
    ps, name = calculate_price_to_sales(symbol)
    g_lower = 0.14
    g_upper = 0.22
    
    calculate_time_premium_payed(beta=ps, g_lower=g_lower, g_upper=g_upper, name=name)

NVIDIA Corporation has a PriceToSalesTTM of 20.94
The revenue of NVIDIA Corporation trades at a premium of 14 years forward at a average growth rate of 18.00 %.


<details>
    <summary>Comment:</summary>
    Because the solution at the top of the notebook and the last ones differ a bit from each other, there must be a strong sensitivity to changes in the input, that the time at a growth rate of 18% is not equal to the average time of a growth rate confidence interval with a mean of 18%.
    This underlines why earnings reactions on big surprises have such significant effect on the traded share price.
</details>